In [7]:
import os
import csv
import glob
import json
import os
import glob
import numpy as np
import pandas as pd
import math
from scipy.signal import argrelmax
import matplotlib.pyplot as plt
import copy
import cv2
def Lp(x,cut=10):               
    dt = 0.01                       # サンプリング間隔
    fn = 1/(2*dt)                   
    fp =cut                         # 通過域端周波数[Hz]
    fs =fp*1.5                      # 阻止域端周波数[Hz]
    gpass = 3                       # 通過域最大損失量[dB]
    gstop = 12                      # 阻止域最小減衰量[dB]
    # 正規化
    Wp = fp/fn
    Ws = fs/fn
    N, Wn = signal.buttord(Wp, Ws, gpass, gstop)#バターワースフィルタ作成
    b1, a1 = signal.butter(N, Wn, "low")#バターワースフィルタ作成
    y1 = signal.filtfilt(b1, a1, x)#バターワースフィルタ実行
    return y1


In [68]:
another_side= [6,8,9,12,14,15,17,19,25,26,31,32,34,35,36,37]####右側麻痺の被験者
sub=list(range(1,38))###被験者番号一覧
sub=[5]
AFO_kind="no-AFO"###処理する条件が
dfdata_all=[]###最終的なデータセットの入れ物
all_gait_each_sub=[]###計測できた歩行周期数を記録


for sub_number in sub:

    ### 処理後のjasonファイルのある場所に移動
    place="/home/brlab/Desktop/oosato_V_step/result2/sub-"+str(sub_number)+"/sub-"+str(sub_number)+"_"+AFO_kind
    os.chdir(place)
    ########################################

    #####もし2試行めがない場合できてしまった余計なファイル削除
    place2=place+"/"+str(2)
    try:
        os.rmdir(place2)
    except:
        dummy=0
    ##################################      
    
    cycle=glob.glob("*")###この条件で何試行したのかを取得

    
    cycle=cycle[0]

    start_second=0 ###動画に人が写った最初の時間
    end_second=1400 ###動画に人が写った最後の時間

    gait_each_sub=0###歩行周期をカウント1始まり
    cycle_count=1###歩行周期をカウント1始まり
    for cycle_number in cycle:
        
        #####人が写っていないときはjson_loadでエラーが出るのでエラーから動画に人が写った最初の時間と動画に人が写った最後の時間を検出
        place="/home/brlab/Desktop/oosato_V_step/result2/sub-"+str(sub_number)+"/sub-"+str(sub_number)+"_"+AFO_kind
        os.chdir(place) 
        jason_count=-1            
        jason_count_list=[]
        jason_count_list_post=[]
        jason_count_list_pre=[]
        print(place)
        os.chdir(cycle_number)
        datatype=["x","y","w"]*25
        dfl= pd.DataFrame(datatype,columns=["jason"])
        jsonnameR=glob.glob('*.json')##全部のjsonファイル名を取得
        jsonname=sorted(jsonnameR)###並び替えて正しい順番に
        for name in jsonname:
            json_open = open(name, 'r')
            json_load = json.load(json_open)
            jason_count=jason_count+1
            try:
                 data=json_load['people'][0]['pose_keypoints_2d']
            except:
                jason_count_list.append(jason_count)


        if jason_count_list!=0:
            for i in range (len(jason_count_list)-1):
                if jason_count_list[i+1]-jason_count_list[i]>200:
                    end_second=jason_count_list[i+1]-1
                    start_second=jason_count_list[i]+1
                    break
                if i == len(jason_count_list)-2:

                    if  jason_count_list[0]<10:
                        start_second=max(jason_count_list)+1
                        end_second=1450


                    else:
                        start_second=0
                        end_second=min((jason_count_list))-1




        else:
            print("##############################################error")
            end_second=1000
            start_second=0

        ##################################################################################################################


        t=0
        
        ###ssub=17 pring1は上のプログラムだと余計な部分も入ってしまうので手入力##################
        if sub_number==17 and AFO_kind=="spring1":
            start_second=start_second+100
        jsonnameR=glob.glob('*.json')##全部のjsonファイル名を取得
        jsonname=sorted(jsonnameR)[start_second:end_second-1]###並び替えて正しい順番に
        ##############################################################################
        
        
        ######とりあえず人jasonデータをCSVにまとめる(確認用#################)
        for name in jsonname:
            json_open = open(name, 'r')
            json_load = json.load(json_open)

            data=json_load['people'][0]['pose_keypoints_2d']    
            df= pd.DataFrame(data,columns=[t])
            dfl=pd.concat([dfl,df], axis=1)
            t=t+0.01



        json_open.close()

        dfl.to_csv("元データ.csv", mode='w')    
        ##########################################################################


        ####全データから必要なキーポイント抜き出し#######################################
        numberlist=[1,8,9,10,11,19,21,22,24,23,0,13,14,12,20]####使用するキーポイントの番号　この番号が何を示すかはOpenPoseのページで

        xlist=[] ###1,8,9,10,11,19,21,22,24 のxざひょう
        ylist=[] ##1,8,9,10,11,19,21,22,24 のyざひょう

        dfmc = pd.read_csv("元データ.csv", engine="python")
        dfmc=dfmc.replace(0, np.nan)
        for i in numberlist:
            xdata=dfmc.iloc[3*i,:]
            xdata= xdata.values.tolist()
            xlist.append(xdata)

            ydata=dfmc.iloc[3*i+1,:]
            ydata= ydata.values.tolist()
            ylist.append(ydata)
        ####################################################################################


        nan_front_list=[]

        for key in range(len(xlist)):
            for i in range (2,60):
                if np.isnan(xlist[key][i]) :
                    nan_front_list.append(i)



        if len(nan_front_list)!=0:
            while np.isnan(xlist[0][max(nan_front_list)+1]) or  np.isnan(xlist[1][max(nan_front_list)+1]) or np.isnan(xlist[2][max(nan_front_list)+1]) or np.isnan(xlist[3][max(nan_front_list)+1]) or np.isnan(xlist[4][max(nan_front_list)+1]) or np.isnan(xlist[5][max(nan_front_list)+1]) or np.isnan(xlist[6][max(nan_front_list)+1]) or np.isnan(xlist[7][max(nan_front_list)+1]) or np.isnan(xlist[8][max(nan_front_list)+1]) or np.isnan(xlist[9][max(nan_front_list)+1]) or np.isnan(xlist[10][max(nan_front_list)+1]) or np.isnan(xlist[11][max(nan_front_list)+1]) or np.isnan(xlist[12][max(nan_front_list)+1]) or np.isnan(xlist[13][max(nan_front_list)+1]) :
                nan_front_list.append(max(nan_front_list)+1)



        nan_back_list=[]
        for key in range(len(xlist)):
            for i in range (1,int(len(xlist[key])/10)):
                if np.isnan(xlist[key][-i]):
                    nan_back_list.append(i)

        if len(nan_back_list)!=0:
            while np.isnan(xlist[0][-max(nan_back_list)-1]) or  np.isnan(xlist[1][-max(nan_back_list)-1]) or np.isnan(xlist[2][-max(nan_back_list)-1]) or np.isnan(xlist[3][-max(nan_back_list)-1]) or np.isnan(xlist[4][-max(nan_back_list)-1]) or np.isnan(xlist[5][-max(nan_back_list)-1]) or np.isnan(xlist[6][-max(nan_back_list)-1]) or np.isnan(xlist[7][-max(nan_back_list)-1]) or np.isnan(xlist[8][-max(nan_back_list)-1]) or np.isnan(xlist[9][-max(nan_back_list)-1]) or np.isnan(xlist[10][-max(nan_back_list)-1]) or np.isnan(xlist[11][-max(nan_back_list)-1]) or np.isnan(xlist[12][-max(nan_back_list)-1]) or np.isnan(xlist[13][-max(nan_back_list)-1]) :
                nan_back_list.append(max(nan_back_list)+1)

        if len(nan_front_list)!=0:
            nan_front=max(nan_front_list)
        else:
            nan_front=2

        if len(nan_back_list)!=0:
            nan_back=max(nan_back_list)
        else:
            nan_back=0

        for i in range(len(xlist)):
            xlist[i]=xlist[i][nan_front+1:-nan_back-1]
            ylist[i]=ylist[i][nan_front+1:-nan_back-1]
        start_second=start_second+nan_front+1
        end_second=end_second-nan_back-1


        for i in range(len(xlist)):
            for j in range(2,42):
                if abs(xlist[i][j+1]-xlist[i][j])>100:
                    xlist[i][j+1]=np.nan


        t=0
        timerec=[]
        for i in range (len(xlist[0])-2):
            timerec.append(t)
            t=t+0.01
        dfl= pd.DataFrame(timerec,columns=["t"])


        for i in range (len(xlist)):
            df= pd.DataFrame(xlist[i],columns=[numberlist[i]])
            dfl=pd.concat([dfl,df], axis=1)


        df=dfl.interpolate('spline', order=3)    


        xlist=[]
        for i in range(len(numberlist)):
            xdata=df.iloc[:,i+1]
            xdata= xdata.values.tolist()
            xlist.append(xdata)



        t=0
        timerec=[]
        for i in range (len(ylist[0])-2):
            timerec.append(t)
            t=t+0.01
        dfl= pd.DataFrame(timerec,columns=["t"])


        for i in range (len(ylist)):
            df= pd.DataFrame(ylist[i],columns=[numberlist[i]])
            dfl=pd.concat([dfl,df], axis=1)
        df=dfl.interpolate('spline', order=3)    

        ylist=[]
        for i in range(len(numberlist)):
            ydata=df.iloc[:,i+1]
            ydata= ydata.values.tolist()
            ylist.append(ydata)



        ####かかとの移動から左右の入れ替わりを検知
        low_lib_change=[]
        min_L=xlist[6][100]
        max_R=xlist[8][100]
        max_R_list=[xlist[8][100]]
        min_L_list=[xlist[6][100]]
        for i in range(100,len(xlist[8])-100):
            if sub_number in another_side:
                if xlist[8][i]-max_R<-30:
                    low_lib_change.append(i)
                else:
                    if xlist[8][i]-max_R<30:
                        if xlist[8][i+1]-xlist[8][i]>-5:
                            max_R=xlist[8][i]
                    max_R_list.append(max_R)

            else:
                if min_L-xlist[6][i]<-30:
                    low_lib_change.append(i)
                else:
                    if min_L-xlist[6][i]<30:
                        if xlist[6][i]-xlist[6][i+1]>-5 :
                            min_L=xlist[6][i]
                min_L_list.append(min_L)



        if len(low_lib_change)!=0:

            Rlist=[3,4,7,8,9]
            Llist=[11,12,5,6,14]

            xlist_temp=copy.deepcopy(xlist)
            ylist_temp=copy.deepcopy(ylist)

            for sec in low_lib_change:
                for j in range(len(Rlist)):
                    xlist_temp[Rlist[j]][sec]=xlist[Llist[j]][sec]
                    xlist_temp[Llist[j]][sec]=xlist[Rlist[j]][sec]
                    ylist_temp[Rlist[j]][sec]=ylist[Llist[j]][sec]
                    ylist_temp[Llist[j]][sec]=ylist[Rlist[j]][sec]

            ylist=ylist_temp
            xlist=xlist_temp  
            
        for i in range(len(xlist)):
            xlist[i]=Lp(xlist[i],4)
            ylist[i]=Lp(ylist[i],4)
            
        Video_name="/home/brlab/Desktop/oosato/症例/sub-{}/動画/{}/*.avi".format(sub_number,AFO_kind)
        print(sorted(glob.glob(Video_name))[0])
        

        #VideoCapture オブジェクトを取得
        cap = cv2.VideoCapture(sorted(glob.glob(Video_name))[0])
        print(os.getcwd())
        speed=0.1
        #動画のプロパティを取得
        width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        fps = cap.get(cv2.CAP_PROP_FPS)
        fpss = fps*speed
        

#         #書き出し設定
        fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        writer = cv2.VideoWriter('{1}x_{0}'.format("tmp.mp4",speed), fourcc, fpss, (int(width), int(height)))
        i=0
        os.makedirs("/home/brlab/Desktop/{}_{}_img".format(sub_number,AFO_kind),exist_ok=True)
        Op_cou=0
        while True:
            i=i+1
            ret, frame = cap.read()
            cv2.imwrite("/home/brlab/Desktop/{}_{}_img/{}.png".format(sub_number,AFO_kind,str(i).zfill(4)), frame)


            if i>= start_second and i<end_second:
                cv2.putText(frame,text='Time={}'.format(i),
                org=(0, 30),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1.0,
                color=(0, 255, 0),
                thickness=2,
                lineType=cv2.LINE_4)
                for key in range(len(xlist)):
                    cv2.circle(frame, (int(xlist[key][Op_cou]),int(ylist[key][Op_cou]) ), 5, (255, 255, 255), thickness=-1)
                
                cv2.imwrite("/home/brlab/Desktop/{}_{}_img/{}.png".format(sub_number,AFO_kind,str(i).zfill(4)), frame)
#                 plt.imshow(frame)
#                 plt.show()
                writer.write(frame)
                Op_cou=Op_cou+1
                
                

            if i>=end_second:
                cv2.imwrite("/home/brlab/Desktop/{}_{}_img/{}.png".format(sub_number,AFO_kind,str(i).zfill(4)), frame)
            if i==end_second:
                break
writer.release()
cap.release()
cv2.destroyAllWindows()
   
            

/home/brlab/Desktop/oosato_V_step/result2/sub-5/sub-5_no-AFO
/home/brlab/Desktop/oosato/症例/sub-5/動画/no-AFO/01.avi
/home/brlab/Desktop/oosato_V_step/result2/sub-5/sub-5_no-AFO/1
